In [0]:
print("this is a testing code for notebook")

this is a testing code for notebook


In [0]:
%pip install pydeequ

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
import os
os.environ["SPARK_VERSION"] = "3.5"

In [0]:
from pyspark.sql import SparkSession
from pydeequ.checks import Check, CheckLevel
from pydeequ.verification import VerificationSuite, VerificationResult

# SparkSession is already available as `spark` in Databricks

# Sample data
data = spark.createDataFrame([
    (1, 100),
    (2, 200),
    (3, 300),
    (4, None)
], ["transaction_id", "sales_amount"])

# Define checks
check = Check(spark, CheckLevel.Error, "Data quality checks") \
    .isNonNegative("sales_amount")

# Run verification
result = VerificationSuite(spark).onData(data).addCheck(check).run()

# Convert results to DataFrame
result_df = VerificationResult.checkResultsAsDataFrame(spark, result)
result_df.show(truncate=False)

# Overall status
print("Verification status:", result.status)


+-------------------+-----------+------------+----------------------------------------------------------------------------------------------------------------------------+-----------------+------------------+
|check              |check_level|check_status|constraint                                                                                                                  |constraint_status|constraint_message|
+-------------------+-----------+------------+----------------------------------------------------------------------------------------------------------------------------+-----------------+------------------+
|Data quality checks|Error      |Success     |ComplianceConstraint(Compliance(sales_amount is non-negative,COALESCE(CAST(sales_amount AS DECIMAL(20,10)), 0.0) >= 0,None))|Success          |                  |
+-------------------+-----------+------------+----------------------------------------------------------------------------------------------------------------------

/databricks/spark/python/pyspark/sql/dataframe.py:163: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [0]:
from pydeequ.checks import Check, CheckLevel
from pydeequ.verification import VerificationSuite
from pyspark.sql import SparkSession
import uuid

# Spark session (already available in Databricks)
spark = SparkSession.builder.appName("DataQualityChecks").getOrCreate()

# Load sample data with schema inference
# data = spark.read.format("csv") \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .load("dbfs:/FileStore/shared_uploads/traininguser8@sudosu.ai/sample_sales_data-1.csv")

data = spark.table("hive_metastore.default.sample_sales_data")

# Define quality rules
check = Check(spark, CheckLevel.Error, "Data quality checks") \
    .hasSize(lambda x: x >= 100) \
    .isNonNegative("sales_amount")

# Run verification
result = VerificationSuite(spark) \
    .onData(data) \
    .addCheck(check) \
    .run()

result_df = VerificationResult.checkResultsAsDataFrame(spark, result)

unique_id = str(uuid.uuid4())
result_df.write.mode("overwrite").json(f"dbfs:/FileStore/deequ_report_{unique_id}")

# Stop job if failed
if result.status != "Success":
    print("Deequ checks failed.")
else:
    print("Deequ checks passed")


/databricks/spark/python/pyspark/sql/dataframe.py:163: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


Deequ checks failed.


In [0]:
report_path = f"dbfs:/FileStore/deequ_report_{unique_id}"
report_df = spark.read.json(report_path)
report_df.show(truncate=False)
report_df.write.mode("overwrite").saveAsTable("deequ_report_results")

+-------------------+-----------+------------+----------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------+-----------------+
|check              |check_level|check_status|constraint                                                                                                                  |constraint_message                                 |constraint_status|
+-------------------+-----------+------------+----------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------+-----------------+
|Data quality checks|Error      |Error       |ComplianceConstraint(Compliance(sales_amount is non-negative,COALESCE(CAST(sales_amount AS DECIMAL(20,10)), 0.0) >= 0,None))|                                                   |Success          |
|Data quality checks|Error      

In [0]:
%fs ls dbfs:/FileStore/

path,name,size,modificationTime
dbfs:/FileStore/deequ_report_0d476116-0dbd-497b-bd68-a6b18b3c13fb/,deequ_report_0d476116-0dbd-497b-bd68-a6b18b3c13fb/,0,1755070222000
dbfs:/FileStore/deequ_report_1a016e02-b673-44fd-9361-3c48c38e1199/,deequ_report_1a016e02-b673-44fd-9361-3c48c38e1199/,0,1755071856000
dbfs:/FileStore/deequ_report_412491f1-53e6-4cfd-80cf-a8a2c80e8011/,deequ_report_412491f1-53e6-4cfd-80cf-a8a2c80e8011/,0,1755071878000
dbfs:/FileStore/deequ_report_54499761-4875-445c-8011-98fa26ff3f20/,deequ_report_54499761-4875-445c-8011-98fa26ff3f20/,0,1755070267000
dbfs:/FileStore/deequ_report_64cef1ff-a432-4292-a778-6ac7e42ce487/,deequ_report_64cef1ff-a432-4292-a778-6ac7e42ce487/,0,1755071266000
dbfs:/FileStore/deequ_report_efa5e9fa-2599-4a3a-9d6f-f84fb95ef791/,deequ_report_efa5e9fa-2599-4a3a-9d6f-f84fb95ef791/,0,1755070234000
dbfs:/FileStore/deequ_report_f311ea48-a0d4-44f6-b611-2367262ac880/,deequ_report_f311ea48-a0d4-44f6-b611-2367262ac880/,0,1755071354000
dbfs:/FileStore/shared_uploads/,shared_uploads/,0,1755067723000
